In [ ]:
# 1. Create or append 'countries' table
countries_unique = cities_df["country"].drop_duplicates()
country_table = pd.DataFrame({"country": countries_unique})
existing_countries = pd.read_sql("countries", con=connection_string)
country_table = country_table[~country_table['country'].isin(existing_countries['country'])]
country_table.to_sql('countries', con=connection_string, if_exists='append', index=False)

# 2. Create or append 'cities' table
country_from_sql = pd.read_sql("countries", con=connection_string)
city_table = cities_df.merge(country_from_sql, on="country", how="left")
existing_cities = pd.read_sql("cities", con=connection_string)
city_table = city_table[~city_table['city'].isin(existing_cities['city'])]
city_table = city_table.drop(columns=["country", "population", "population_year"])
city_table = city_table[["city", "country_id", "latitude", "longitude"]]
city_table.to_sql('cities', con=connection_string, if_exists='append', index=False)

# 3. Create or append 'populations' table
cities_from_sql = pd.read_sql("cities", con=connection_string)
population_table = cities_from_sql.merge(cities_df, on="city", how="left")
existing_populations = pd.read_sql("populations", con=connection_string)
population_table = population_table[~(population_table['city_id'].isin(existing_populations['city_id']) & population_table['population_year'].isin(existing_populations['population_year']))]
population_table = population_table.drop(columns=['city', 'latitude_x', 'longitude_x', 'country_id', 'country', 'latitude_y', 'longitude_y'])
population_table.to_sql('populations', con=connection_string, if_exists='append', index=False)

# 4. Create or append 'weather' table
cities_from_sql = pd.read_sql("cities", con=connection_string)
weather_table = weather_df.merge(cities_from_sql, on="city", how="left")
weather_table = weather_table.drop(columns=["city", "country_id", "latitude", "longitude"])
weather_table = weather_table.dropna(subset=["city_id"])
existing_weather = pd.read_sql("weather", con=connection_string)
weather_table = weather_table[~weather_table['city_id'].isin(existing_weather['city_id'])]
weather_table.to_sql('weather', con=connection_string, if_exists='append', index=False)

# 5. Create or append 'airports' table
cities_from_sql = pd.read_sql("cities", con=connection_string)
airports_table = airports_df.merge(cities_from_sql, on="city", how="left")
airports_table = airports_table.drop(columns=["city", "latitude", "longitude", "country_id"])
airports_table = airports_table.rename(columns={"name": "airportName", "location.lat": "latitude", "location.lon": "longitude"})
airports_table = airports_table.drop_duplicates(subset=["icao"])
existing_airports = pd.read_sql("SELECT icao FROM airports", con=connection_string)
airports_table = airports_table[~airports_table['icao'].isin(existing_airports['icao'])]
airports_table.to_sql('airports', con=connection_string, if_exists='append', index=False)

# 6. Create or append 'city_airport' table
cities_from_sql = pd.read_sql("SELECT city_id FROM cities", con=engine)
airports_from_sql = pd.read_sql("SELECT airport_id, city_id FROM airports", con=engine)
city_airport_table = airports_from_sql.merge(cities_from_sql, on="city_id", how="inner")
city_airport_table = city_airport_table.dropna(subset=["city_id", "airport_id"])
existing_city_airport = pd.read_sql("SELECT city_id, airport_id FROM airport_city", con=engine)
city_airport_table = city_airport_table[~city_airport_table.set_index(['city_id', 'airport_id']).index.isin(existing_city_airport.set_index(['city_id', 'airport_id']).index)]
city_airport_table.to_sql('airport_city', con=engine, if_exists='append', index=False)

# 7. Create or append 'flights' table
# INNER JOIN to process only airports with actual flight data. This prevents empty rows in the flights table.
airports_from_sql = pd.read_sql("airports", con=connection_string)
flights_table = airports_from_sql.merge(flights_df, left_on="iata", right_on="AirportName", how="inner")
flights_table = flights_table.drop(columns=['icao', 'iata', 'timeZone', 'airportName', 'shortName', 'municipalityName', 'countryCode', 'latitude', 'longitude', 'city_id', 'AirportName'])
existing_flights = pd.read_sql("flights", con=connection_string)
flights_table = flights_table[~(flights_table['FlightNumber'].isin(existing_flights['FlightNumber']) & flights_table['ScheduledArrival'].isin(existing_flights['ScheduledArrival']))]
flights_table.to_sql('flights', con=connection_string, if_exists='replace', index=False)